In [13]:
import pandas as pd

In [3]:
df = pd.read_csv("/Users/arnaud/workspaces/linalgo/newspapers-clustering/data/newsclust.csv")

In [8]:
df = df.query("site != 'cbn.com'")

In [41]:
from bias import Bias

df['bias'] = df.apply(lambda row: Bias.get_bias_for_domain(row['site']), axis=1).astype('str')

In [42]:
TEST_DOMAINS =  {
    'bloomberg.com',
     'breitbart.com',
     'c-span.org',
     'chicagotribune.com',
     'chron.com'
}

In [43]:
df_train =  df.loc[~df['site'].isin(TEST_DOMAINS)]
df_test =  df.loc[df['site'].isin(TEST_DOMAINS)]

In [45]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df['bias'].values)

X_train = df_train['text'].values
y_train = encoder.transform(df_train['bias'].values)

X_test = df_test['text'].values
y_test = encoder.transform(df_test['bias'].values)

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [63]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('class', LogisticRegression())
])

In [64]:
%%time

pipeline.fit(X_train, y_train)

CPU times: user 2min 4s, sys: 1.38 s, total: 2min 5s
Wall time: 2min 5s


Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [69]:
df_test['pred'] = encoder.inverse_transform(pipeline.predict(X_test))

/Users/arnaud/anaconda/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
df_test[['title', 'site', 'bias', 'pred']].head()

,title,site,bias,pred
23,Assembly Republicans launch their election cam...,chron.com,Bias.LEFT_CENTER,Bias.LEFT_CENTER
35,Thursday's Scores - Houston Chronicle,chron.com,Bias.LEFT_CENTER,Bias.LEFT_CENTER
37,Saturday's Scores - Houston Chronicle,chron.com,Bias.LEFT_CENTER,Bias.RIGHT_CENTER
43,Ingraham: GOP Establishment Now 'Firmly Entren...,breitbart.com,Bias.RIGHT,Bias.RIGHT
47,About the Chicago Tribune&apos;s suburban site...,chicagotribune.com,Bias.LEFT_CENTER,Bias.LEFT


In [84]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Precision = %s" % precision_score(df_test['bias'], df_test['pred'], average='macro'))
print("Recall = %s" % accuracy_score(df_test['bias'], df_test['pred']))

Precision = 0.207067586376
Recall = 0.495676905575
